In [1]:
# import all the modules
from nipype.pipeline.engine import Workflow, Node, MapNode, JoinNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, FreeSurferSource
from nipype.interfaces.fsl import FAST, Reorient2Std, Merge, MeanImage
from nipype.interfaces.freesurfer import FSCommand, MRIConvert, Binarize
from nipype.interfaces.ants import RegistrationSynQuick, ApplyTransforms, CorticalThickness
from pandas import DataFrame, Series, read_csv
from glob import glob

# FSL setup- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

#freesurfer setup
fs_dir = '/moochie/Cat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

# Study specific variables
analysis_home = '/home/camachocm2/Analysis/aggregate_anats'

workflow_dir = analysis_home + '/workflows'
proc_dir = analysis_home + '/proc/subj_data'
group_dir = analysis_home + '/proc/group_data'

subject_info = read_csv(analysis_home + '/misc/subject_info.csv', index_col=None)
all_subjects = subject_info['freesurferID'].tolist()
#all_subjects = ['101']

template_subjects_list = analysis_home + '/misc/template_subs.txt'
template_subjects = open(template_subjects_list).read().splitlines()
tissue_subjects = ['236','2020','2007','105','109','112','113','125','223','1010']

#set up datasink
substitutions = [('_subject_id_','')]
datasink = Node(DataSink(substitutions=substitutions,
                         base_directory = proc_dir),
                name = 'datasink')

## 1. Template Brain Creation
This section creates a template representative of the sample based on a predetermined list.

In [ ]:
##### Set up nodes for file handling #####

template_source = MapNode(FreeSurferSource(subjects_dir = fs_dir), 
                          name = 'template_source', 
                          iterfield = ['subject_id'])
template_source.inputs.subject_id = template_subjects

In [ ]:
######### Template creation functions #########
def make3DTemplate(subject_T1s, num_proc, output_prefix):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from os.path import abspath, split
    from os import getcwd
    from shutil import copyfile
    from glob import glob
    from subprocess import check_call

    curr_dir = getcwd()

    #copy T1s into current directory
    for T in range(0,len(subject_T1s)):
        [dirname,filename] = split(subject_T1s[T])
        copyfile(subject_T1s[T],curr_dir + '/S' + str(T)+'_'+filename)

    # -c flag is control for local computing (2= use localhost; required for -j flag)
    # -j flag is for number of processors allowed
    check_call(['antsMultivariateTemplateConstruction2.sh', '–d','3','–o', output_prefix,'–r','1','–c','2','–j', str(num_proc), '*.nii.gz'])
    
    sample_template = abspath(output_prefix + 'template0.nii.gz')
    
    return(sample_template)


In [ ]:
######### Template creation nodes #########

#convert freesurfer brainmask files to .nii
convertT1 = Node(MRIConvert(out_file='T1.nii.gz',
                            out_type='niigz'), 
                 name='convertT1')

#reorient files to standard space
reorientT1 = MapNode(Reorient2Std(),
                     name = 'reorientT1',
                     iterfield = ['in_file'])

#pass files into template function (normalized, pre-skull-stripping)
makeTemplate = Node(Function(input_names=['subject_T1s','num_proc','output_prefix'],
                             output_names=['sample_template'],
                             function=make3DTemplate),
                    name='makeTemplate')
makeTemplate.inputs.num_proc=4 
makeTemplate.inputs.output_prefix='child_'

# skullstrip template brain
strip_template = Node(BET(), name='strip_template')

In [ ]:
######### Template creation workflow #########
template_flow = Workflow(name = 'template_flow')
template_flow.connect([(template_source, convertT1, [('T1','in_file')]),
                       (convertT1, reorientT1, [('out_file', 'in_file')]),
                       (reorientT1, makeTemplate, [('out_file', 'subject_T1s')]),
                       (makeTemplate, strip_template,[('sample_template','in_file')]),
                       
                       (strip_template, datasink, [('out_file','skullstrip_template')]),
                       (reorientT1, datasink, [('out_file','proc_T1s')]),
                       (makeTemplate, datasink, [('sample_template', 'sample_template')]),
                      ])

template_flow.base_dir = workflow_dir
template_flow.write_graph(graph2use = 'flat')
template_flow.run(plugin='MultiProc',plugin_args={'memory_gb':10})

## 2. Tissue Segmentation
This next section creates the 4 tissue segmentations plus a brainmask for cortical thickness estimation tissue priors:
1. CSF 
2. cortical gray matter 
3. Subcortical gray matter
4. White matter   
5. Brainstem
6. Cerebellum

In [ ]:
## data handling nodes

# subject handling node
infosource = Node(IdentityInterface(fields=['subject_id']), 
                  name='infosource')
infosource.iterables = [('subject_id', template_subjects)]


#pull freesurfer data
tissue_source = Node(FreeSurferSource(subjects_dir = fs_dir),
                     name = 'tissue_source')

In [ ]:
## Tissue labeling functions

def relabel_fast(fast_tissue_list):
    from nipype import config, logging
    from os.path import split
    from os import rename
    config.enable_debug_mode()
    logging.update_logging(config)
    tissue_list = sorted(fast_tissue_list)
    csf = tissue_list[0]
    wm = tissue_list[2]
    [wd, csf_file] = split(csf)
    [wd, wm_file] = split(wm)
    rename(csf, wd + 'csf.nii.gz')
    rename(wm, wd + 'wm.nii.gz')
    wm_csf = [wd + 'csf.nii.gz', wd + 'wm.nii.gz']
    return(wm_csf)

# Create subcortical and cortical gray matter masks <-- custom function. inputs: fs aseg + tissue class files
def aseg_to_tissuemaps(aseg):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros_like
    from os.path import abspath
    aseg_nifti = load(aseg)
    aseg_data = aseg_nifti.get_data()
    cortical_labels = [3, 42]
    subcortical_labels =[10, 11, 12, 13, 17, 18, 26, 49, 50, 51, 52, 53, 54, 58, 28, 60]
    brainstem_labels = [16]
    cerebellum_labels = [6, 7, 8, 45, 46, 47]

    #creating array of zeroes that replaces 0's with 1's when matches values of subcortical_labels
    cortical_data = zeros_like(aseg_data)
    for x in cortical_labels:
        cortical_data[aseg_data == x] = 1
    cortical_nifti = Nifti1Image(cortical_data, aseg_nifti.affine)
    
    subcort_data = zeros_like(aseg_data) 
    for x in subcortical_labels:
        subcort_data[aseg_data == x] = 1
    subcort_nifti = Nifti1Image(subcort_data, aseg_nifti.affine)
    
    bs_data = zeros_like(aseg_data) 
    for x in brainstem_labels:
        bs_data[aseg_data == x] = 1
    bs_nifti = Nifti1Image(bs_data, aseg_nifti.affine)
    
    cb_data = zeros_like(aseg_data) 
    for x in cerebellum_labels:
        cb_data[aseg_data == x] = 1
    cb_nifti = Nifti1Image(cb_data, aseg_nifti.affine)
    
    save(subcort_nifti, 'subcortical_gm.nii.gz')
    save(cortical_nifti, 'cortical_gm.nii.gz')
    save(bs_nifti, 'brainstem.nii.gz')
    save(cb_nifti, 'cerebellum.nii.gz')
    subcort_file = abspath('subcortical_gm.nii.gz')
    cortical_file = abspath('cortical_gm.nii.gz')
    brainstem_file = abspath('brainstem.nii.gz')
    cerebellum_file = abspath('cerebellum.nii.gz')
    gm_list = [subcort_file, cortical_file, brainstem_file, cerebellum_file]
    return(gm_list)

def combine_seg(file_list_1, file_list_2):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros_like
    from os.path import abspath

    in_files = file_list_1 + file_list_2

    for tissue in in_files:
        if 'segmentcsf' in tissue:
            csf_file = tissue
            csf_nifti = load(csf_file)
            csf_data = csf_nifti.get_data()
        elif 'subcortical' in tissue:
            subcortical_file = tissue
            sc_nifti = load(subcortical_file)
            sc_data = sc_nifti.get_data()
        elif 'cortical' in tissue:
            cortex_file = tissue
            cort_nifti = load(cortex_file)
            cort_data = cort_nifti.get_data()
        elif 'segmentwm' in tissue:
            wm_file = tissue
            wm_nifti = load(wm_file)
            wm_data = wm_nifti.get_data()
        elif 'brainstem' in tissue:
            brainstem_file = tissue
            bs_nifti = load(brainstem_file)
            bs_data = bs_nifti.get_data()
        elif 'cerebellum' in tissue:
            cerebellum_file = tissue
            cb_nifti = load(cerebellum_file)
            cb_data = cb_nifti.get_data()

    combined_labels_data = zeros_like(csf_data)

    # these are done in a deliberate order, later tissues overwrite previous ones
    combined_labels_data[csf_data>0] = 1
    combined_labels_data[wm_data>0] = 4
    combined_labels_data[bs_data>0] = 5
    combined_labels_data[sc_data>0] = 3
    combined_labels_data[cb_data>0] = 6
    combined_labels_data[cort_data>0] = 2

    combined_labels_nii = Nifti1Image(combined_labels_data, csf_nifti.affine)
    combined_labels_file = save(combined_labels_nii, 'combined_tissues.nii.gz')
    tissue_map = abspath('combined_tissues.nii.gz')
    
    return(tissue_map)

In [ ]:
## tissue labeling nodes

#convert freesurfer brainmask files to .nii
convert_to_nii = Node(MRIConvert(out_file='brain.nii.gz',
                                 out_type='niigz'), 
                      name='convert_to_nii')

#reorient brainmask file to standard
reorient_to_std = Node(Reorient2Std(),
                       name = 'reorient_to_std')

#convert freesurfer T1 file to .nii
convert_T1 = Node(MRIConvert(out_file='brain.nii.gz',
                             out_type='niigz'),
                  name='convert_T1')

#reorient brainmask file to standard
reorient_T1 = Node(Reorient2Std(),
                   name = 'reorient_T1')

#binarize the freesurfer brainmask so it's a true mask
binarize_bm = Node(Binarize(min=20, dilate=1, erode=1), 
                   name='binarize_bm')

# Reorient aseg to standard
reorient_aseg = Node(Reorient2Std(),
                     name = 'reorient_aseg')

#T1 gets run through segmentation (2) ---> results in segmentation into 3 tissue classes (wm, gm, csf)
segment = Node(FAST(number_classes = 3, 
                    segments=True, 
                    no_bias=True), 
               name = 'segment')

# Convert freesurfer aseg to nii
convert_aseg = Node(MRIConvert(out_file='aseg.nii.gz',
                               out_type='niigz'), 
                    name='convert_aseg')

# Split aseg into to types of gray matter
aseg_to_gm = Node(Function(input_names=['aseg'],
                           output_names=['gm_list'],
                           function=aseg_to_tissuemaps),
                  name='aseg_to_gm')

# Relabel the FAST segmentation 
relabel_fast_seg = Node(Function(input_names=['fast_tissue_list'],
                                 output_names=['wm_csf'],
                                 function=relabel_fast),
                        name='relabel_fast_seg')

#combine tissue segmentation to 1 volume for easier editing of tissue priors
combine_labels = Node(Function(input_names=['file_list_1', 'file_list_2'], 
                               output_names=['tissue_map'], 
                               function=combine_seg),
                      name='combine_labels')

In [ ]:
######### Tissue segmentation workflow #########
segment_flow = Workflow(name = 'segment_flow')
segment_flow.connect([(infosource, tissue_source, [('subject_id','subject_id')]),
                      (tissue_source, convert_to_nii, [('brainmask','in_file')]),
                      (convert_to_nii, reorient_to_std, [('out_file', 'in_file')]),
                      (reorient_to_std, segment, [('out_file', 'in_files')]),
                      (segment, relabel_fast_seg, [('tissue_class_files', 'fast_tissue_list')]),
                      (tissue_source, convert_aseg, [('aseg','in_file')]),
                      (convert_aseg, reorient_aseg, [('out_file', 'in_file')]),
                      (reorient_aseg, aseg_to_gm, [('out_file', 'aseg')]),
                      (aseg_to_gm, combine_labels, [('gm_list','file_list_1')]),
                      (relabel_fast_seg, combine_labels, [('wm_csf','file_list_2')]),
                      (tissue_source, convert_T1, [('T1','in_file')]),
                      (convert_T1, reorient_T1, [('out_file','in_file')]),
                      (reorient_to_std, binarize_bm, [('out_file','in_file')]),
                      
                      (binarize_bm, datasink, [('binary_file','brain_mask')]),
                      (reorient_T1, datasink, [('out_file','subject_T1')]),
                      (combine_labels, datasink, [('tissue_map','combined_labels')])
                     ])

segment_flow.base_dir = workflow_dir
segment_flow.write_graph(graph2use = 'flat')
segment_flow.run('MultiProc', plugin_args={'n_procs': 1, 'memory_gb':4})

## 3. Tissue Priors
This section creates the tissue priors (after manual editing the outputs from the previous step)

In [2]:
### Data handling nodes
template_brain = proc_dir + '/sample_template/lcbd_template0.nii.gz'
priors_source_files = {'tissue_map': proc_dir + '/combined_labels/{subject_id}/combined_tissues.nii.gz',
                       'T1': proc_dir + '/subject_T1/{subject_id}/brain_reoriented.nii.gz', 
                       'brainmask': proc_dir + '/brain_mask/{subject_id}/brainmask_reoriented_thresh.nii.gz'}
priors_files = Node(SelectFiles(priors_source_files), name='template_files')
priors_files.iterables = [('subject_id',tissue_subjects)]

In [3]:
def split_tissue_labels(combined_labels):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros_like
    from os.path import abspath
    
    labeled_nifti = load(combined_labels)
    labeled_data = labeled_nifti.get_data()
    
    segment1_data = zeros_like(labeled_nifti.get_data())
    segment1_data[labeled_data==1] = 1
    segment1_img = Nifti1Image(segment1_data,labeled_nifti.affine)
    save(segment1_img, 'tissue_class1.nii.gz')
    segment1 = abspath('tissue_class1.nii.gz')
    
    segment2_data = zeros_like(labeled_nifti.get_data())
    segment2_data[labeled_data==2] = 1    
    segment2_img = Nifti1Image(segment2_data, header=labeled_nifti.header, affine=labeled_nifti.affine)
    save(segment2_img, 'tissue_class2.nii.gz')
    segment2 = abspath('tissue_class2.nii.gz')
    
    segment3_data = zeros_like(labeled_nifti.get_data())
    segment3_data[labeled_data==3] = 1
    segment3_img = Nifti1Image(segment3_data, header=labeled_nifti.header, affine=labeled_nifti.affine)
    save(segment3_img, 'tissue_class3.nii.gz')
    segment3 = abspath('tissue_class3.nii.gz')
    
    segment4_data = zeros_like(labeled_nifti.get_data())
    segment4_data[labeled_data==4] = 1
    segment4_img = Nifti1Image(segment4_data, header=labeled_nifti.header, affine=labeled_nifti.affine)
    save(segment4_img, 'tissue_class4.nii.gz')
    segment4 = abspath('tissue_class4.nii.gz')
    
    segment5_data = zeros_like(labeled_nifti.get_data())
    segment5_data[labeled_data==5] = 1
    segment5_img = Nifti1Image(segment5_data, header=labeled_nifti.header, affine=labeled_nifti.affine)
    save(segment5_img, 'tissue_class5.nii.gz')
    segment5 = abspath('tissue_class5.nii.gz')
    
    segment6_data = zeros_like(labeled_nifti.get_data())
    segment6_data[labeled_data==6] = 1
    segment6_img = Nifti1Image(segment6_data, header=labeled_nifti.header, affine=labeled_nifti.affine)
    save(segment6_img, 'tissue_class6.nii.gz')
    segment6 = abspath('tissue_class6.nii.gz')
    
    tissues = [segment1,segment2,segment3,segment4,segment5,segment6]
    return(tissues)

In [ ]:
### Tissue priors nodes
split_tissues = Node(Function(input_names=['combined_labels'],
                              output_names=['tissues'],
                              function=split_tissue_labels), 
                     name='split_tissues')
# register to template
reg_to_template = Node(RegistrationSynQuick(fixed_image=template_brain),
                       name = 'reg_to_template')

# apply the transform to the tissue maps
applyxform = MapNode(ApplyTransforms(reference_image=template_brain), name = 'applyxform', iterfield=['input_image'])

# apply the transform to the tissue maps
applyxformBM = Node(ApplyTransforms(reference_image=template_brain), name = 'applyxformBM')

# merge brainmasks together
mergeBM = JoinNode(Merge(dimension='t'), name='mergeBM', joinsource='priors_files',joinfield='in_files')

# average brain masks to make brain prior
avg_brainmasks = Node(MeanImage(dimension='T',output_datatype='double',out_file='brain_prior.nii.gz'), 
                      name='avg_brainmasks')

In [ ]:
tissuepriors_flow = Workflow(name = 'tissuepriors_flow')
tissuepriors_flow.connect([(priors_files, reg_to_template, [('T1', 'moving_image')]),
                           (reg_to_template, applyxform, [('out_matrix','transforms')]),
                           (reg_to_template, applyxformBM, [('out_matrix','transforms')]),
                           (priors_files, split_tissues, [('tissue_map','combined_labels')]), 
                           (split_tissues, applyxform, [('tissues','input_image')]),
                           (priors_files, applyxformBM, [('brainmask','input_image')]),
                           
                           (applyxformBM, datasink, [('output_image','warped_brainmasks')]),
                           (reg_to_template,datasink, [('warped_image','registered_T1')]),
                           (applyxform, datasink, [('output_image','warped_tissues')])
                          ])
tissuepriors_flow.base_dir = workflow_dir
tissuepriors_flow.write_graph(graph2use = 'flat')
tissuepriors_flow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

# average the tissue maps
merge_tissues = Merge(dimension='t')
avg_tissues = MeanImage(dimension='T',output_datatype='double')
from glob import glob
from os import mkdir

mkdir(proc_dir+'/tissue_priors')

for a in range(0,6):
    files = glob(proc_dir+'/warped_tissues/*/_applyxform{0}/tissue_class{1}_trans.nii.gz'.format(a,a+1))
    merge_tissues.inputs.in_files = files
    merge_tissues.inputs.merged_file = proc_dir+'/warped_tissues/merged_tissue{0}.nii.gz'.format(a)
    merge_tissues.run()
   
    avg_tissues.inputs.in_file = proc_dir+'/warped_tissues/merged_tissue{0}.nii.gz'.format(a)
    avg_tissues.inputs.out_file = proc_dir+'/tissue_priors/tissue{0}.nii.gz'.format(a)
    avg_tissues.run()
    
files = glob(proc_dir+'/warped_brainmasks/*/brainmask_reoriented_thresh_trans.nii.gz')
merge_tissues.inputs.in_files = files
merge_tissues.inputs.merged_file = proc_dir+'/warped_tissues/merged_brainmasks.nii.gz'
merge_tissues.run()

avg_tissues.inputs.in_file = proc_dir+'/warped_tissues/merged_brainmasks.nii.gz'
avg_tissues.inputs.out_file = proc_dir+'/tissue_priors/brainmask_prior.nii.gz'
avg_tissues.run()

## 4. Calculate Cortical Thickness
This section now takes all these items and calculates cortical thickness per subject. 

In [5]:
def combine_transforms(SubjectToTemplate1Warp,SubjectToTemplate0GenericAffine):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    transforms = [SubjectToTemplate1Warp, SubjectToTemplate0GenericAffine]
    
    return(transforms)

In [6]:
# Pull participant data from the FreeSurfer directory
direct_info = Node(IdentityInterface(fields=['subject_id']), 
                   name='direct_info')
direct_info.iterables = [('subject_id',all_subjects)]

direct_source = Node(FreeSurferSource(subjects_dir=fs_dir), 
                     name='direct_source')

# set up cortical thickness estimation for the whole sample
priors = glob(proc_dir+'/tissue_priors/tissue*.nii.gz')
priors = sorted(priors)

direct = Node(CorticalThickness(brain_probability_mask=proc_dir+'/tissue_priors/brainmask_prior.nii.gz', 
                                brain_template=proc_dir + '/sample_template/lcbd_template0.nii.gz', 
                                segmentation_priors=priors, 
                                t1_registration_template=proc_dir+'/skullstrip_template/lcbd_template_stripped.nii.gz', 
                                num_threads=3), 
              name='direct')

combine_xfms = Node(Function(input_names=['SubjectToTemplate1Warp','SubjectToTemplate0GenericAffine'], 
                             output_names=['transforms'], 
                             function=combine_transforms), 
                    name='combine_xfms')

template_xfm = Node(ApplyTransforms(reference_image=proc_dir + '/sample_template/lcbd_template0.nii.gz',
                                    interpolation='NearestNeighbor'), 
                    name='template_xfm')

In [ ]:
direct_flow = Workflow(name='direct_flow')
direct_flow.connect([(direct_info, direct_source,[('subject_id','subject_id')]), 
                     (direct_source, direct, [('T1','anatomical_image')]),
                     (direct, template_xfm, [('CorticalThickness','input_image')]),
                     (direct, combine_xfms, [('SubjectToTemplate1Warp','SubjectToTemplate1Warp'),
                                             ('SubjectToTemplate0GenericAffine','SubjectToTemplate0GenericAffine')]),
                     (combine_xfms, template_xfm, [('transforms','transforms')]),
                     (direct, datasink, [('BrainVolumes','CT_BrainVolumes'), 
                                         ('BrainSegmentation','CT_BrainSegmentation'), 
                                         ('CorticalThickness','CT_CorticalThickness'),
                                         ('SubjectToTemplate0GenericAffine','CT_SubjectToTemplate0GenericAffine'),
                                         ('SubjectToTemplate1Warp','CT_SubjectToTemplate1Warp')]),
                     (template_xfm,datasink,[('output_image','Final_CT_templateSpace')])
                    ])
direct_flow.base_dir = workflow_dir
direct_flow.write_graph(graph2use = 'flat')
direct_flow.run('MultiProc', plugin_args={'n_procs': 27, 'memory_gb':40})

190311-08:52:23,90 workflow INFO:
	 Generated workflow graph: /home/camachocm2/Analysis/aggregate_anats/workflows/direct_flow/graph.png (graph2use=flat, simple_form=True).
190311-08:52:23,318 workflow INFO:
	 Workflow direct_flow settings: ['check', 'execution', 'logging', 'monitoring']
190311-08:52:24,224 workflow INFO:
	 Running in parallel.
190311-08:52:24,301 workflow INFO:
	 [MultiProc] Running 0 tasks, and 176 jobs ready. Free memory (GB): 40.00/40.00, Free processors: 27/27.
190311-08:52:24,470 workflow INFO:
	 [Node] Setting-up "direct_flow.direct_source" in "/home/camachocm2/Analysis/aggregate_anats/workflows/direct_flow/_subject_id_204/direct_source".
190311-08:52:24,502 workflow INFO:
	 [Node] Setting-up "direct_flow.direct_source" in "/home/camachocm2/Analysis/aggregate_anats/workflows/direct_flow/_subject_id_2025/direct_source".
190311-08:52:24,538 workflow INFO:
	 [Node] Running "direct_source" ("nipype.interfaces.io.FreeSurferSource")
190311-08:52:24,563 workflow INFO:
	